In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable
import matplotlib as mpl
from matplotlib.ticker import AutoLocator, AutoMinorLocator, LogLocator
from matplotlib.colors import Normalize
import glob
from scipy.interpolate import griddata
from pathlib import Path
import h5py
import sys
from matplotlib.lines import Line2D
from matplotlib.legend_handler import HandlerTuple

# Where am I running?
try:
    # Normal script
    here = Path(__file__).resolve().parent
except NameError:
    # Notebook / REPL
    here = Path.cwd()

phys_const_path = (here / '..' / 'phys_const').resolve()
sys.path.append(str(phys_const_path))

nsm_plots_path = (here / '..' / 'nsm_plots').resolve()
sys.path.append(str(nsm_plots_path))

nsm_plots_postproc = (here / '..' / 'nsm_instabilities').resolve()
sys.path.append(str(nsm_plots_postproc))

import phys_const as pc
import plot_functions as pf
import functions_angular_crossings as fac

### Reading hdf5 data

In [ ]:
dir_last = '/home/erick/software/devscrpts/gw170817_paper_plots/nsmcell_15_48_15_doma_48-1-1cm_cells_192-1-1_att_1_dt_1e-12s_novac_nomat_sinus_plt1500000_particles_angular_distributions.h5'

with h5py.File(dir_last, "r") as hf:

    # Read ELN angular distribution
    phi_eln_xln_fi_last = hf["phi_eln_xln_fi"][:]
    mu_eln_xln_fi_last = hf["mu_eln_xln_fi"][:]
    G_eln_xln_fi_last = hf["G_eln_xln_fi"][:]
    phi_eln_xln_fp_last = hf["phi_eln_xln_fp"][:]
    mu_eln_xln_fp_last = hf["mu_eln_xln_fp"][:]
    G_eln_xln_fp_last = hf["G_eln_xln_fp"][:]

    # Read e (nu_e) angular distribution
    phi_e_fi_last = hf["phi_e_fi"][:]
    mu_e_fi_last = hf["mu_e_fi"][:]
    G_e_fi_last = hf["G_e_fi"][:]
    phi_e_fp_last = hf["phi_e_fp"][:]
    mu_e_fp_last = hf["mu_e_fp"][:]
    G_e_fp_last = hf["G_e_fp"][:]

    # Read ebar (anti-nu_e) angular distribution
    phi_ebar_fi_last = hf["phi_ebar_fi"][:]
    mu_ebar_fi_last = hf["mu_ebar_fi"][:]
    G_ebar_fi_last = hf["G_ebar_fi"][:]
    phi_ebar_fp_last = hf["phi_ebar_fp"][:]
    mu_ebar_fp_last = hf["mu_ebar_fp"][:]
    G_ebar_fp_last = hf["G_ebar_fp"][:]

    # Read u (nu_x) angular distribution
    phi_u_fi_last = hf["phi_u_fi"][:]
    mu_u_fi_last = hf["mu_u_fi"][:]
    G_u_fi_last = hf["G_u_fi"][:]
    phi_u_fp_last = hf["phi_u_fp"][:]
    mu_u_fp_last = hf["mu_u_fp"][:]
    G_u_fp_last = hf["G_u_fp"][:]

dir_init = '/home/erick/software/devscrpts/gw170817_paper_plots/nsmcell_15_48_15_doma_48-1-1cm_cells_192-1-1_att_1_dt_1e-12s_novac_nomat_sinus_plt00000_particles_angular_distributions.h5'

with h5py.File(dir_init, "r") as hf:

    # Read ELN angular distribution
    phi_eln_xln_fi_init = hf["phi_eln_xln_fi"][:]
    mu_eln_xln_fi_init = hf["mu_eln_xln_fi"][:]
    G_eln_xln_fi_init = hf["G_eln_xln_fi"][:]
    phi_eln_xln_fp_init = hf["phi_eln_xln_fp"][:]
    mu_eln_xln_fp_init = hf["mu_eln_xln_fp"][:]
    G_eln_xln_fp_init = hf["G_eln_xln_fp"][:]

    # Read e (nu_e) angular distribution
    phi_e_fi_init = hf["phi_e_fi"][:]
    mu_e_fi_init = hf["mu_e_fi"][:]
    G_e_fi_init = hf["G_e_fi"][:]
    phi_e_fp_init = hf["phi_e_fp"][:]
    mu_e_fp_init = hf["mu_e_fp"][:]
    G_e_fp_init = hf["G_e_fp"][:]

    # Read ebar (anti-nu_e) angular distribution
    phi_ebar_fi_init = hf["phi_ebar_fi"][:]
    mu_ebar_fi_init = hf["mu_ebar_fi"][:]
    G_ebar_fi_init = hf["G_ebar_fi"][:]
    phi_ebar_fp_init = hf["phi_ebar_fp"][:]
    mu_ebar_fp_init = hf["mu_ebar_fp"][:]
    G_ebar_fp_init = hf["G_ebar_fp"][:]

    # Read u (nu_x) angular distribution
    phi_u_fi_init = hf["phi_u_fi"][:]
    mu_u_fi_init = hf["mu_u_fi"][:]
    G_u_fi_init = hf["G_u_fi"][:]
    phi_u_fp_init = hf["phi_u_fp"][:]
    mu_u_fp_init = hf["mu_u_fp"][:]
    G_u_fp_init = hf["G_u_fp"][:]

In [ ]:
def plot_colored_lines_grid(y_list, color_maps, titles, filename,
                            ylowerlimit=None, yupperlimit=None):
    fig, axs = plt.subplots(2, 4, figsize=(18, 6),
                            sharex=True, sharey=True,
                            gridspec_kw=dict(hspace=0.0, wspace=0.0))
    axs = axs.flatten()

    for i, y in enumerate(y_list):

        ax = axs[i]

        c = ax.pcolormesh(y[3], y[4], y[5], shading='auto', cmap=color_maps[i], vmin=ylowerlimit[i], vmax=yupperlimit[i])
        ax.scatter(y[0], y[1], c=y[2], cmap=color_maps[i], vmin=ylowerlimit[i], vmax=yupperlimit[i], s=8, edgecolor='gray')

        # ax.text(0.03, 0.15, titles[i], transform=ax.transAxes, fontsize=22,
        #     verticalalignment='top', horizontalalignment='left',
        #     bbox=dict(facecolor='none', alpha=0.7, edgecolor='none'))

        # Set horizontal axis in angles from 0 to 2pi, showing integers of pi and half fractions
        ax.set_xlim(0, 2 * np.pi)
        ax.set_xticks([0.5 * np.pi, 1.0 * np.pi, 1.5 * np.pi])
        ax.set_xticklabels([r'$\frac{1}{2}\pi$', r'$\pi$', r'$\frac{3}{2}\pi$'])
        ax.set_ylim(-1,1)
        ax.tick_params(axis='both', which='both', direction='in', top=True, right=True)
        ax.xaxis.set_minor_locator(AutoMinorLocator())
        ax.yaxis.set_minor_locator(AutoMinorLocator())
    
    # Shared axis labels
    fig.text(0.5, 0.02, r'$\phi$', ha='center', fontsize=22)
    fig.text(0.07, 0.5, r'$\cos \theta$', va='center', rotation='vertical', fontsize=22)

    # --- Colorbars per column, on top with ticks/labels above ---
    for col in range(4):  # three columns

        norm = Normalize(ylowerlimit[col], vmax=yupperlimit[col])
        cmap = plt.cm.get_cmap(color_maps[col])
        sm = plt.cm.ScalarMappable(cmap=cmap, norm=norm)
        sm.set_array([])
        # Position: [left, bottom, width, height] in figure coords
        left = 0.128 + col * (0.188+0.005)
        bottom = 0.92  # put high at top
        width = 0.188
        height = 0.03
        cax = fig.add_axes([left, bottom, width, height])
        cbar = fig.colorbar(sm, cax=cax, orientation='horizontal')

        # Put ticks + label above
        cbar.ax.xaxis.set_ticks_position('top')
        cbar.ax.xaxis.set_label_position('top')
        cbar.set_label(titles[col], fontsize=18, labelpad=8)
        cbar.ax.xaxis.set_minor_locator(AutoMinorLocator())

    # Save and show
    if filename:
        plt.savefig(filename, bbox_inches='tight')
    plt.show()
    plt.close(fig)

a=0
b=-485
c=None

plot_colored_lines_grid(
    y_list = [
        [phi_eln_xln_fi_init, mu_eln_xln_fi_init, G_eln_xln_fi_init/1e9, phi_eln_xln_fp_init, mu_eln_xln_fp_init, G_eln_xln_fp_init/1e9],
        [phi_e_fi_init, mu_e_fi_init,             G_e_fi_init      /1e9, phi_e_fp_init, mu_e_fp_init,             G_e_fp_init      /1e9],
        [phi_ebar_fi_init, mu_ebar_fi_init,       G_ebar_fi_init   /1e9, phi_ebar_fp_init, mu_ebar_fp_init,       G_ebar_fp_init   /1e9],
        [phi_u_fi_init, mu_u_fi_init,             G_u_fi_init      /1e9, phi_u_fp_init, mu_u_fp_init,             G_u_fp_init     /1e9],
        [phi_eln_xln_fi_last, mu_eln_xln_fi_last, G_eln_xln_fi_last/1e9, phi_eln_xln_fp_last, mu_eln_xln_fp_last, G_eln_xln_fp_last/1e9],
        [phi_e_fi_last, mu_e_fi_last,             G_e_fi_last      /1e9, phi_e_fp_last, mu_e_fp_last,             G_e_fp_last     /1e9],
        [phi_ebar_fi_last, mu_ebar_fi_last,       G_ebar_fi_last   /1e9, phi_ebar_fp_last, mu_ebar_fp_last,       G_ebar_fp_last  /1e9],
        [phi_u_fi_last, mu_u_fi_last,             G_u_fi_last      /1e9, phi_u_fp_last, mu_u_fp_last,             G_u_fp_last     /1e9],
    ],
    color_maps=[
        'seismic', 'Reds', 'Blues', 'Greens',
        'seismic', 'Reds', 'Blues', 'Greens',],
    titles = [
        r'$G_{\mathrm{ELN-XLN}}\,\left[10^9\,\mathrm{s}^{-1}\,\mathrm{Sr}^{-1}\right]$', 
        r'$G_{\nu_e}\,\left[10^9\,\mathrm{s}^{-1}\,\mathrm{Sr}^{-1}\right]$', 
        r'$G_{\bar{\nu}_e}\,\left[10^9\,\mathrm{s}^{-1}\,\mathrm{Sr}^{-1}\right]$', 
        r'$G_{\nu_\mu}\,\left[10^9\,\mathrm{s}^{-1}\,\mathrm{Sr}^{-1}\right]$', 
    ],
    filename = f"plots/angular_distribution.png",
    ylowerlimit = np.array([-4.51833597e+09,1.46257479e+09,6.71444161e+08,1.86834413e+06,-4.51833597e+09,1.46257479e+09,6.71444161e+08,1.86834413e+06])/1e9,
    yupperlimit = np.array([+4.51833597e+09,9.52227218e+09,1.21471491e+10,+4.36328775e+09,+4.51833597e+09,9.52227218e+09,1.21471491e+10,+4.36328775e+09])/1e9
)